In [1]:
# Import Libraries
from pyspark.sql.types import StructType, StructField, FloatType, BooleanType
from pyspark.sql.types import DoubleType, IntegerType, StringType
import pyspark
from pyspark import SQLContext
from pyspark.sql import SparkSession
import os 

In [2]:
import sys
from pyspark import SparkContext, SparkConf

In [4]:
  # create Spark context with Spark configuration
  conf = SparkConf().setAppName("Spark Count").setMaster("spark://node-master:7077")
  sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/03 10:10:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
df = sc.textFile('hdfs://node-master:8020/user/hadoop/WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [10]:
ty

AttributeError: 'RDD' object has no attribute 'show'

In [8]:
# get threshold
threshold = 1

# read in text file and split each document into words
tokenized = sc.textFile('hdfs://node-master:8020/user/hadoop/WA_Fn-UseC_-HR-Employee-Attrition.csv').flatMap(lambda line: line.split(" "))

# count the occurrence of each word
wordCounts = tokenized.map(lambda word: (word, 1)).reduceByKey(lambda v1,v2:v1 +v2)

# filter out words with fewer than threshold occurrences
filtered = wordCounts.filter(lambda pair:pair[1] >= threshold)

# count characters
charCounts = filtered.flatMap(lambda pair:pair[0]).map(lambda c: c).map(lambda c: (c, 1)).reduceByKey(lambda v1,v2:v1 +v2)

list = charCounts.collect()
print(repr(list)[1:-1])

('i', 793779), ('j', 15774), ('P', 48326), ('(', 31737), ('è', 1), ('n', 741910), ('%', 3463), ('\x02', 11), ('\\', 19495), ('7', 984206), ('{', 5016), ('S', 84980), ('L', 47965), ('e', 1251503), ('"', 209678), ('<', 3439), ('8', 984606), ('5', 989950), ('X', 4210), ('v', 105627), ('[', 11572), (';', 80121), ('}', 5825), ('\t', 484), ('’', 227), ('V', 16253), ('#', 1957), ('\x12', 9), ('|', 4027), ('/', 13220), ('R', 56383), (']', 20923), ('b', 170062), ('G', 45177004), ('a', 875316), ('“', 208), ('E', 72589), ('Y', 12668), ('h', 348378), ('C', 45105511), (':', 115817), (',', 365553), ('”', 374), ('_', 41176), ('‘', 1), ('^', 601), ('`', 12306), ('\x1d', 2), ('-', 656777), ('W', 23802), ('\x07', 1), ('I', 61268), ('K', 17108), ('Q', 3120), ('6', 986903), ('\x01', 27), ('\x1c', 19), ('—', 117), ('c', 342213), ('.', 350250), ('t', 698195), ('=', 4022), ("'", 153128), ('Z', 4662), ('f', 145659), ('\x03', 1), ('9', 982617), ('H', 43309), ('\x14', 1), ('l', 533048), ('u', 363641), ('O', 432

In [19]:
list = wordCounts.collect()
print(repr(list)[1:500])

('', 6909207), ('yours', 1634), ('sweeten', 69), ('solve', 408), ('companions,', 774), ('ploughs', 70), ('up,"', 511), ('Questionings', 1), ('strangers', 1290), ('destroyed', 2220), ('ablaze,', 19), ('voluble', 92), ('reign.', 207), ('intractable', 52), ('flower."', 17), ('Latona,', 8), ('elements.', 138), ('diminished,', 126), ("glar'd", 3), ('fiercer', 125), ('iv.', 870), ('treatise', 364), ('atrocities', 61), ('Son.', 102), ('Broken,', 5), ("overpower'd.", 1), ('fardel', 16), ("ebb'd", 3), (


In [9]:
lines = sc.textFile('hdfs://node-master:8020/user/hadoop/D1GB/*')
counts = lines.flatMap(lambda x: x.split(' ')) \
                .map(lambda x: (x, 1)) \
                .reduceByKey(lambda x, y:x+y)
output = counts.collect()
lines = []

for (word, count) in output:
    line = word + ": " +str(count)
    lines.append(line)
    
rdd = sc.parallelize(lines)
rdd.saveAsTextFile('/home/tam/result')

Py4JJavaError: An error occurred while calling o102.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory hdfs://node-master:8020/user/hadoop/spark_wc/result already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:299)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:406)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.Pai

In [14]:
rdd.saveAsTextFile('/home/tam/result')

23/01/30 10:27:41 WARN TaskSetManager: Stage 6 contains a task of very large size (43798 KiB). The maximum recommended task size is 1000 KiB.
23/01/30 10:27:41 ERROR Inbox: An error happened while processing message in the inbox for CoarseGrainedScheduler
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3537)
	at java.base/java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:100)
	at java.base/java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:132)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1897)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:1806)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1203)
	at java.base/java.io.ObjectOutputStream.writeObject(ObjectOutputStream.java:358)
	at org.

Exception in thread "dispatcher-CoarseGrainedScheduler" java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3537)
	at java.base/java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:100)
	at java.base/java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:132)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1897)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:1806)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1203)
	at java.base/java.io.ObjectOutputStream.writeObject(ObjectOutputStream.java:358)
	at org.apache.spark.serializer.JavaSerializationStream.writeObject(JavaSerializer.scala:46)
	at org.apache.spark.serializer.JavaSerializerInstance.serialize(JavaSerializer.scala:115)
	at org.apache.spark.sch

23/01/30 10:27:46 WARN TaskSetManager: Lost task 6.0 in stage 6.0 (TID 6660) (192.168.39.139 executor 1): java.io.IOException: Mkdirs failed to create file:/home/tam/result/_temporary/0/_temporary/attempt_202301301027405582904230318978078_0030_m_000006_0 (exists=false, cwd=file:/home/hadoop/spark/work/app-20230130100920-0000/1)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1081)
	at org.apache.hadoop.mapred.TextOutputFormat.getRecordWriter(TextOutputFormat.java:113)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.initWriter(SparkHadoopWriter.scala:238)
	at org.apache.spark.internal.io.SparkHadoopWriter$.executeTask(SparkHadoopWriter.scala:126)
	at org.apache.spark.internal.io.SparkHadoopWriter$.$anonfun$write$1(SparkHadoopWriter.scala:88)


23/01/30 10:27:50 WARN TaskSetManager: Lost task 6.1 in stage 6.0 (TID 6662) (192.168.39.139 executor 1): java.io.IOException: Mkdirs failed to create file:/home/tam/result/_temporary/0/_temporary/attempt_202301301027405582904230318978078_0030_m_000006_1 (exists=false, cwd=file:/home/hadoop/spark/work/app-20230130100920-0000/1)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1081)
	at org.apache.hadoop.mapred.TextOutputFormat.getRecordWriter(TextOutputFormat.java:113)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.initWriter(SparkHadoopWriter.scala:238)
	at org.apache.spark.internal.io.SparkHadoopWriter$.executeTask(SparkHadoopWriter.scala:126)
	at org.apache.spark.internal.io.SparkHadoopWriter$.$anonfun$write$1(SparkHadoopWriter.scala:88)


23/01/30 10:27:55 WARN TaskSetManager: Lost task 6.2 in stage 6.0 (TID 6664) (192.168.39.139 executor 1): java.io.IOException: Mkdirs failed to create file:/home/tam/result/_temporary/0/_temporary/attempt_202301301027405582904230318978078_0030_m_000006_2 (exists=false, cwd=file:/home/hadoop/spark/work/app-20230130100920-0000/1)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1081)
	at org.apache.hadoop.mapred.TextOutputFormat.getRecordWriter(TextOutputFormat.java:113)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.initWriter(SparkHadoopWriter.scala:238)
	at org.apache.spark.internal.io.SparkHadoopWriter$.executeTask(SparkHadoopWriter.scala:126)
	at org.apache.spark.internal.io.SparkHadoopWriter$.$anonfun$write$1(SparkHadoopWriter.scala:88)


23/01/30 10:27:59 WARN TaskSetManager: Lost task 6.3 in stage 6.0 (TID 6666) (192.168.39.139 executor 1): java.io.IOException: Mkdirs failed to create file:/home/tam/result/_temporary/0/_temporary/attempt_202301301027405582904230318978078_0030_m_000006_3 (exists=false, cwd=file:/home/hadoop/spark/work/app-20230130100920-0000/1)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1081)
	at org.apache.hadoop.mapred.TextOutputFormat.getRecordWriter(TextOutputFormat.java:113)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.initWriter(SparkHadoopWriter.scala:238)
	at org.apache.spark.internal.io.SparkHadoopWriter$.executeTask(SparkHadoopWriter.scala:126)
	at org.apache.spark.internal.io.SparkHadoopWriter$.$anonfun$write$1(SparkHadoopWriter.scala:88)


Py4JJavaError: An error occurred while calling o136.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:106)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:406)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$4(PairRDDFunctions.scala:1062)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScop

23/01/30 10:27:59 WARN TaskSetManager: Lost task 7.3 in stage 6.0 (TID 6667) (192.168.39.139 executor 1): TaskKilled (Stage cancelled)


In [15]:
dataColl=rdd.collect()
for row in dataColl:
    print(row[0] + "," +str(row[1]))

23/01/30 10:31:01 WARN TaskSetManager: Stage 7 contains a task of very large size (43798 KiB). The maximum recommended task size is 1000 KiB.
23/01/30 10:31:02 ERROR Inbox: An error happened while processing message in the inbox for CoarseGrainedScheduler
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3537)
	at java.base/java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:100)
	at java.base/java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:132)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.base/java.io.DataOutputStream.write(DataOutputStream.java:112)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1873)
	at java.base/java.io.ObjectOutputStream.write(ObjectOutputStream.java:723)
	at java.base/java.nio.channels.Channels$WritableByteChannelImpl.write(Channels.java:392)
	at org.apache.spark.util.SerializableBuffer.$

Exception in thread "dispatcher-CoarseGrainedScheduler" java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3537)
	at java.base/java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:100)
	at java.base/java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:132)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.base/java.io.DataOutputStream.write(DataOutputStream.java:112)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1873)
	at java.base/java.io.ObjectOutputStream.write(ObjectOutputStream.java:723)
	at java.base/java.nio.channels.Channels$WritableByteChannelImpl.write(Channels.java:392)
	at org.apache.spark.util.SerializableBuffer.$anonfun$writeObject$1(SerializableBuffer.scala:49)
	at org.apache.spark.util.SerializableBuffer$$Lambda$1424/0x00000008018bd788.apply(Unknown Source)
	at org.apache.spark.util.Utils$.tryOrIOException(

KeyboardInterrupt: 